In [ ]:
!pip install numpy==1.19.5
!pip install pandas==1.3.1  
!pip install tensorflow-gpu==2.5.0
!pip install tensorflow-datasets==4.4.0
!pip install tensorflow-hub==0.12.0  
!pip install giotto-tda==0.5.1
!pip install keras==2.7.0 
!pip install keras-flops==0.1.2
!pip install keras-tcn==3.3.0
!pip install matplotlib==3.5.0
!pip install jupyterlab==3.0.16
!pip install torch==1.10.0             
!pip install torchvision==0.11.1
!pip install tqdm==4.61.2
!pip install pytorch2keras==0.2.4 
!pip install onnx==1.8.0              
!pip install onnx-tf==1.8.0
!pip install onnx2keras==0.0.24
!pip install librosa==0.8.1
!pip install kapre==0.2.0
!pip install scikit-learn==0.24.2             
!pip install scipy==1.7.0 

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2) 
import tensorflow_datasets as tfds
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
inputShape = (224, 224)
preprocess = imagenet_utils.preprocess_input

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
import numpy as np
from tensorflow.keras.utils import to_categorical
import cv2

In [ ]:
dataset_name = "cifar10"
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name, split=["train", "test"], with_info=True, as_supervised=True
)
NUM_CLASSES = ds_info.features["label"].num_classes

IMG_SIZE = 224
batch_size = 1
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label


ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

In [ ]:
model_path = "" #@param {type:"string"}
model  = tf.keras.models.load_model(model_path)


In [ ]:
iterations = 7
alpha = 2
epsilon = 8/255  

In [ ]:
#pgd_generation
image_count = 0
flag = 0
orig_label = []
for x_train, labels in ds_train:
  gen_img = tf.identity(x_train)
  gen_img = gen_img + tf.random.uniform(gen_img.get_shape().as_list(), minval=-epsilon, maxval=epsilon, dtype=tf.dtypes.float32)
  x_temp = x_train.numpy()
  for iters in range(iterations):
    imgv = tf.Variable(gen_img)
    with tf.GradientTape() as tape:
      tape.watch(imgv)
      predictions = model(imgv)
      loss = tf.keras.losses.CategoricalCrossentropy()(labels, predictions)
      grads = tape.gradient(loss,imgv)
    signed_grads = tf.sign(grads)
    gen_img = gen_img + (alpha*signed_grads)
    gen_img = tf.clip_by_value(gen_img, x_train-epsilon, x_train+epsilon)
  for i in range(16):
    image_count += 1
    if image_count>200:
      flag = 1
      break
    orig_label.append(labels[i])
    gen_image = gen_img.numpy()[i]*255
    orig_image = x_train.numpy()[i]*255
    path_to_generated_and_original_image= "" #@param {type:"string"}
    imwrite(path_to_generated_and_original_image +str(image_count)+'_gen_img.png',np.clip(gen_image, 0, 255).astype('uint8'))
    imwrite(path_to_generated_and_original_image+str(image_count)+'_orig_img.png',np.clip(orig_image, 0, 255).astype('uint8'))
  if flag==1:
    break 

In [ ]:
generated_images = []
for i in range(1, 201):
  x = cv2.imread(path_to_generated_and_original_image+str(i)+'_gen_img.png')
  generated_images.append(x)
generated_images = tf.convert_to_tensor(generated_images)
orig_label = tf.convert_to_tensor(orig_label)

In [ ]:
original_images = []
for i in range(1, 201):
  x = cv2.imread(path_to_generated_and_original_image+str(i)+'_orig_img.png')
  original_images.append(x)
original_images = tf.convert_to_tensor(generated_images)

In [ ]:
model.evaluate(generated_images,orig_label, verbose = 1) # adversarial accuracy i.e. model accuracy on the perturbed data 